# MeetingScheduler: Find Optimal Times to Meet

Change TZ_CONFIG according to who you want to meet with.

Keys: Timezones of participants

Values: number of people per timezone

In [1]:
# Change according to who you want to meet with
# Timezone: number of people
TZ_CONFIG = {
    "Manila": 2,
    "Oslo": 1,
    "Kolkata": 1,
    "New_York": 1,
    "Los_Angeles": 3,
    "Brisbane": 1,
    "London": 1
}

## Installation

Just run the cells below

In [2]:
!pip install pendulum

In [3]:
import pytz
import pendulum
import pandas as pd

In [4]:
def find_tz(tz):
    try:
        return [s for s in list(pytz.all_timezones) if tz in s][0]
    except IndexError:
        print(f"Cant find {tz}")

def get_time_score(dt):
    hour = dt.hour
    dow = dt.day_of_week
    # bad days: weekends
    if dow in [pendulum.SATURDAY, pendulum.SUNDAY]:
        return -3
    # bad times, weight very low
    if hour < 6:
        return -2
    if hour > 22:
        return -2
    # regular 9 to 5 schedule is optimal
    if 9 < hour < 17:
        return 1
    else:
        return 0.5

In [5]:
TIMEZONES = []
for k, v in TZ_CONFIG.items():
    for _ in range(v):
        TIMEZONES.append(k)
PROPER_TZS = [find_tz(tz) for tz in TIMEZONES]

In [6]:
PROPER_TZS

['Asia/Manila',
 'Asia/Manila',
 'Europe/Oslo',
 'Asia/Kolkata',
 'America/New_York',
 'America/Los_Angeles',
 'America/Los_Angeles',
 'America/Los_Angeles',
 'Australia/Brisbane',
 'Europe/London']

In [7]:
def find_time(date_str="", tz="Los_Angeles"):
    """Finds best time for people in different timezones to meet.
    
        Args:
            date_str = MM-DD
            tz = which timezones to display the time in
    """
    _tz = find_tz(tz)
    month, day = date_str.split("-")
    year = pendulum.now().year
    base_tz = pendulum.datetime(year, int(month), int(day), 0, 0)
    df = []
    index_time = []
    for _ in range(24):        
        out = {}
        index_time.append(pendulum.from_timestamp(base_tz.timestamp(), _tz).to_datetime_string())
        for i, tz in enumerate(PROPER_TZS):
            tz_dt = pendulum.from_timestamp(base_tz.timestamp(), tz)
            out[tz + str(i)] = get_time_score(tz_dt)
        df.append(out)
        base_tz = base_tz.add(hours=1)
    df = pd.DataFrame(df)
    df.index = index_time
    return df.sum(axis=1).sort_values(ascending=False)

## Find times to meet!

In [8]:
# args: MM-DD, timezone to display (default: PST)
find_time("05-24")

2022-05-23 22:00:00    4.5
2022-05-24 07:00:00    4.0
2022-05-24 06:00:00    3.5
2022-05-23 19:00:00    1.5
2022-05-23 21:00:00    1.5
2022-05-23 18:00:00    0.5
2022-05-24 13:00:00   -0.5
2022-05-24 10:00:00   -0.5
2022-05-24 16:00:00   -1.0
2022-05-23 20:00:00   -1.0
2022-05-24 15:00:00   -1.0
2022-05-24 03:00:00   -1.0
2022-05-24 04:00:00   -1.0
2022-05-24 08:00:00   -1.5
2022-05-24 05:00:00   -1.5
2022-05-24 09:00:00   -2.0
2022-05-23 17:00:00   -2.0
2022-05-24 01:00:00   -3.0
2022-05-24 11:00:00   -3.0
2022-05-24 12:00:00   -3.0
2022-05-23 23:00:00   -3.0
2022-05-24 02:00:00   -3.5
2022-05-24 00:00:00   -3.5
2022-05-24 14:00:00   -3.5
dtype: float64

In [9]:
find_time("05-24", tz="Manila")

2022-05-24 13:00:00    4.5
2022-05-24 22:00:00    4.0
2022-05-24 21:00:00    3.5
2022-05-24 10:00:00    1.5
2022-05-24 12:00:00    1.5
2022-05-24 09:00:00    0.5
2022-05-25 04:00:00   -0.5
2022-05-25 01:00:00   -0.5
2022-05-25 07:00:00   -1.0
2022-05-24 11:00:00   -1.0
2022-05-25 06:00:00   -1.0
2022-05-24 18:00:00   -1.0
2022-05-24 19:00:00   -1.0
2022-05-24 23:00:00   -1.5
2022-05-24 20:00:00   -1.5
2022-05-25 00:00:00   -2.0
2022-05-24 08:00:00   -2.0
2022-05-24 16:00:00   -3.0
2022-05-25 02:00:00   -3.0
2022-05-25 03:00:00   -3.0
2022-05-24 14:00:00   -3.0
2022-05-24 17:00:00   -3.5
2022-05-24 15:00:00   -3.5
2022-05-25 05:00:00   -3.5
dtype: float64